# Python Packaging Essentials — `setup.py` and `pyproject.toml`

*Updated: October 17, 2025*

This hands-on notebook explains **classic `setup.py` packaging** and the **modern `pyproject.toml` (PEP 621)** approach, with examples you can adapt to your own projects.

## Table of Contents
1. What is `setup.py`?
2. Anatomy of a basic `setup.py`
3. Reading dependencies from `requirements.txt`
4. The modern `pyproject.toml` (PEP 621)
5. `src/` layout and package discovery
6. Adding a CLI with console scripts
7. Building distributions (sdist & wheel)
8. Test publishing flow (TestPyPI)
9. Scaffolding a demo project (ready to build locally)
10. Exercises

## 1) What is `setup.py`?

`setup.py` is a build/install script used by **setuptools** to package and distribute Python projects. It contains your project's **metadata**, **dependencies**, and (optionally) **entry points**.  
Although the community is migrating to **`pyproject.toml`**, `setup.py` remains widely recognized and compatible with PyPI.

## 2) Anatomy of a basic `setup.py`

Below is a minimal yet practical example. It reads dependencies from a helper function and uses `find_packages()` to include your packages.

In [ ]:
from setuptools import find_packages, setup
from typing import List

def get_requirements() -> List[str]:
    reqs: List[str] = []
    try:
        with open("requirements.txt") as f:
            for line in f:
                dep = line.strip()
                if dep and dep != "-e .":
                    reqs.append(dep)
    except FileNotFoundError:
        print("requirements.txt not found; continuing without extras")
    return reqs

setup(
    name="sampleproject",
    version="0.0.1",
    description="Demo package for packaging essentials",
    author="Your Name",
    author_email="your@email.com",
    packages=find_packages(),
    install_requires=get_requirements(),
)

## 3) Reading dependencies from `requirements.txt`

Why: keep your dependency list in one place for both **development** and **installation**.

Typical `requirements.txt` example:
```
numpy>=1.26
pandas
-e .
```

The `-e .` line (editable install) should be **ignored** in `install_requires` because it is not a package.

In [ ]:
# Demo: emulate reading requirements (no actual installs here)
from pathlib import Path

demo_req = Path("requirements.txt")
demo_req.write_text("numpy>=1.26\npandas\n-e .\n", encoding="utf-8")

def demo_read_requirements(path="requirements.txt"):
    reqs = []
    for line in Path(path).read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if line and line != "-e .":
            reqs.append(line)
    print("Parsed requirements:", reqs)

demo_read_requirements()

## 4) The modern `pyproject.toml` (PEP 621)

The modern approach uses `pyproject.toml` to declare metadata and the build backend (e.g., setuptools, hatchling, poetry-core).  
**Minimal setuptools example:**

```toml
[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "sampleproject"
version = "0.0.1"
description = "Demo package for packaging essentials"
readme = "README.md"
requires-python = ">=3.9"
authors = [{name="Your Name", email="your@email.com"}]
dependencies = ["numpy>=1.26", "pandas"]
```

## 5) `src/` layout and package discovery

The **`src/` layout** avoids accidental imports from the working directory and makes tests more reliable.

Recommended structure:
```
project/
├─ pyproject.toml
├─ README.md
├─ src/
│  └─ package_name/
│     ├─ __init__.py
│     └─ module.py
└─ tests/
   └─ test_module.py
```

**Setuptools `pyproject.toml` additions for `src/` layout:**
```toml
[tool.setuptools]
package-dir = {"" = "src"}

[tool.setuptools.packages.find]
where = ["src"]
include = ["package_name*"]
```

## 6) Adding a CLI with console scripts

Expose a command-line entry point so users can run your tool directly after installing.

**`pyproject.toml` example:**
```toml
[project.scripts]
mytool = "package_name.cli:main"
```

**`setup.py` example:**
```python
setup(
    ...,
    entry_points={
        "console_scripts": [
            "mytool=package_name.cli:main"
        ]
    }
)
```

## 7) Building distributions (`sdist` & `wheel`)

> These commands run in your terminal (not inside this notebook kernel).

```bash
python -m pip install -U pip build
python -m build
# -> dist/<name>-<version>.tar.gz   (sdist)
# -> dist/<name>-<version>-py3-none-any.whl  (wheel)
```

## 8) Test publishing flow (TestPyPI)

1. Create accounts on **PyPI** and **TestPyPI**, generate API tokens.  
2. Upload to TestPyPI:
```bash
python -m pip install twine
twine upload --repository testpypi dist/*
```
3. Verify installation:
```bash
pip install -i https://test.pypi.org/simple/ sampleproject==0.0.1
```

## 9) Scaffolding a demo project (ready to build locally)

Run the cell below to generate a minimal project under `/mnt/data/demo-packaging/` with:
- `src/` layout
- `pyproject.toml` (setuptools + PEP 621)
- `setup.py` (legacy compatibility)
- a tiny CLI (`hello-cli`) using console scripts

In [ ]:
from pathlib import Path

base = Path("/mnt/data/demo-packaging")
pkg = base / "src" / "hello_pkg"
tests = base / "tests"
base.mkdir(parents=True, exist_ok=True)
pkg.mkdir(parents=True, exist_ok=True)
tests.mkdir(parents=True, exist_ok=True)

# Package files
(pkg / "__init__.py").write_text('__version__ = "0.0.1"\n', encoding="utf-8")
(pkg / "cli.py").write_text('''def main():
    print("Hello from hello-cli!")
''', encoding="utf-8")
(pkg / "module.py").write_text('''def add(a, b):
    return a + b
''', encoding="utf-8")

# Tests
(tests / "test_module.py").write_text('''from hello_pkg.module import add
def test_add():
    assert add(2, 3) == 5
''', encoding="utf-8")

# README & requirements
(base / "README.md").write_text("# hello-pkg\nA demo package.\n", encoding="utf-8")
(base / "requirements.txt").write_text("numpy>=1.26\n", encoding="utf-8")

# pyproject.toml (setuptools + src layout + console script)
(base / "pyproject.toml").write_text('''[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "hello-pkg"
version = "0.0.1"
description = "A demo package"
readme = "README.md"
requires-python = ">=3.9"
authors = [{name="Your Name"}]
dependencies = []

[project.scripts]
hello-cli = "hello_pkg.cli:main"

[tool.setuptools]
package-dir = {"" = "src"}

[tool.setuptools.packages.find]
where = ["src"]
include = ["hello_pkg*"]
''', encoding="utf-8")

# Legacy setup.py
(base / "setup.py").write_text('''from setuptools import setup, find_packages

setup(
    name="hello-pkg",
    version="0.0.1",
    description="A demo package",
    packages=find_packages(where="src"),
    package_dir={"": "src"},
    entry_points={"console_scripts": ["hello-cli=hello_pkg.cli:main"]},
)
''', encoding="utf-8")

print("Scaffolded at:", base)
print("Files:")
for p in base.rglob("*"):
    print(" -", p.relative_to(base))

## 10) Exercises

1. **Add a dependency** to `pyproject.toml` (e.g., `requests`) and rebuild locally with `python -m build`.
2. **Expose another console script** called `hello-plus` that prints `2 + 2` using a function in `hello_pkg.module`.
3. **Switch to a `src/` layout** if you haven't already by adding the `tool.setuptools.*` config (shown above).
4. **Create a `setup.cfg`** with metadata and a minimal `setup.py` stub that only calls `setup()`.
5. **(Advanced)** Publish to **TestPyPI** and confirm installation from the test index.